In [8]:
import pandas as pd
from sklearn.linear_model import Lasso

In [106]:
train = pd.read_csv("30_Training Dataset_V2/training_data.csv")
test = pd.read_csv("30_Public Dataset_Public Sumission Template_v2/public_dataset.csv")
Y_train = train[['單價']]
print(test.shape)

(5876, 21)


In [84]:
con_data = pd.concat([train, test], axis=0)
con_data.shape

(17627, 22)

In [86]:
con_data = con_data.drop(["ID","單價","使用分區",'備註'], axis=1)
con_data = pd.get_dummies(con_data, columns=['縣市','鄉鎮市區','路名','主要用途','主要建材','建物型態'])
con_data.head(5)

,土地面積,移轉層次,總樓層數,屋齡,建物面積,車位面積,車位個數,橫坐標,縱坐標,主建物面積,...,主要建材_其他,主要建材_加強磚造,主要建材_磚造,主要建材_鋼筋混凝土加強磚造,主要建材_鋼筋混凝土造,主要建材_鋼骨造,建物型態_住宅大樓(11層含以上有電梯),建物型態_公寓(5樓含以下無電梯),建物型態_華廈(10層含以下有電梯),建物型態_透天厝
0,-0.256716,11,11,32.583333,-0.174154,-0.819326,0.0,305266,2768378,0.393926,...,0,0,0,0,1,0,1,0,0,0
1,0.100134,7,12,24.166667,0.314204,-0.819326,0.0,300677,2767990,-0.316131,...,0,0,0,0,1,0,1,0,0,0
2,0.181921,10,15,6.166667,0.423366,0.161624,1.0,184815,2504666,-0.098871,...,1,0,0,0,0,0,1,0,0,0
3,0.085594,9,14,8.833333,0.164249,0.524653,1.0,296653,2772355,-0.071147,...,0,0,0,0,1,0,1,0,0,0
4,-0.938116,41,43,11.000000,0.985839,0.532377,1.0,297377,2768472,0.791954,...,0,0,0,0,0,1,1,0,0,0


In [91]:
X_train = con_data[:11751]
X_test = con_data[11751:]

In [98]:
best_alpha = 0.00099

model = Lasso(alpha = best_alpha, max_iter=50000)
model.fit(X_train, Y_train)

Lasso(alpha=0.00099, max_iter=50000)

In [100]:
pred = model.predict(X_test)
pred

array([1.73020793, 1.62858641, 2.63868922, ..., 2.36786258, 3.34305715,
       2.51844869])

In [104]:
result = pd.read_csv("30_Public Dataset_Public Sumission Template_v2/public_submission_template.csv")
result

,ID,predicted_price
0,PU-1,0.0
1,PU-2,0.0
2,PU-3,0.0
3,PU-4,0.0
4,PU-5,0.0
...,...,...
5871,PU-5872,0.0
5872,PU-5873,0.0
5873,PU-5874,0.0
5874,PU-5875,0.0


In [107]:
result['predicted_price'] = pred
result.to_csv("./result.csv", index=False)

In [57]:
X_train = train.drop(["ID","單價","使用分區",'備註'], axis=1)
Y_train = train[["單價"]]
X_train = pd.get_dummies(X_train, columns=['縣市','鄉鎮市區','路名','主要用途','主要建材','建物型態'])

In [58]:
best_alpha = 0.00099

model = Lasso(alpha = best_alpha, max_iter=50000)
model.fit(X_train, Y_train)

Lasso(alpha=0.00099, max_iter=50000)

In [101]:
test = pd.read_csv("30_Public Dataset_Public Sumission Template_v2/public_dataset.csv")
test.head(2)
X_test = test.drop(["ID","使用分區",'備註'], axis=1)
X_test = pd.get_dummies(X_test, columns=['縣市','鄉鎮市區','路名','主要用途','主要建材','建物型態'])

In [108]:
X_test

,土地面積,移轉層次,總樓層數,屋齡,建物面積,車位面積,車位個數,橫坐標,縱坐標,主建物面積,...,主要建材_其他,主要建材_加強磚造,主要建材_磚造,主要建材_鋼筋混凝土加強磚造,主要建材_鋼筋混凝土造,主要建材_鋼骨造,建物型態_住宅大樓(11層含以上有電梯),建物型態_公寓(5樓含以下無電梯),建物型態_華廈(10層含以下有電梯),建物型態_透天厝
0,2.039378,3,7,13.916667,0.472202,0.146176,1.0,288745,2760266,0.849664,...,0,0,0,0,1,0,0,0,1,0
1,-0.077786,5,5,40.833333,-0.546455,-0.819326,0.0,298751,2773492,-0.004102,...,0,0,0,0,1,0,0,1,0,0
2,-0.555766,2,20,1.333333,0.124032,-0.167419,1.0,297000,2769106,-0.285862,...,0,0,0,0,1,0,1,0,0,0
3,-0.744838,6,16,27.500000,-0.027072,0.115280,1.0,293076,2771160,0.027582,...,0,0,0,0,1,0,1,0,0,0
4,-0.610923,3,7,40.500000,-0.863600,-0.819326,0.0,308103,2770220,-0.883328,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5871,0.560113,5,5,0.833333,-0.788335,-0.016028,1.0,166751,2545157,-0.867769,...,0,0,0,0,1,0,0,1,0,0
5872,-1.020142,5,22,3.416667,-0.910137,0.799628,1.0,218814,2672567,-1.107095,...,0,0,0,0,1,0,1,0,0,0
5873,-0.662691,10,12,12.583333,-0.459699,0.004054,1.0,301609,2761359,-0.474267,...,0,0,0,0,1,0,1,0,0,0
5874,0.513752,2,5,36.250000,0.357294,-0.819326,0.0,308833,2770624,0.926327,...,0,0,0,0,1,0,0,1,0,0
